In [1]:
!pip install fastapi kaleido python-multipart uvicorn
!pip install -qU \
  openai==0.27.7 \
  pinecone-client==3.0.0 \
  pinecone-datasets==0.7.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.3 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━

In [2]:
from pinecone_datasets import load_dataset

dataset = load_dataset('langchain-python-docs-text-embedding-ada-002')
# we drop sparse_values as they are not needed for this example
dataset.documents.drop(['metadata'], axis=1, inplace=True)
dataset.documents.rename(columns={'blob': 'metadata'}, inplace=True)
dataset.head()

,id,values,sparse_values,metadata
0.0,417ede5d-39be-498f-b518-f47ed4e53b90,"[0.005949743557721376, 0.01983247883617878, -0...",None,"{'chunk': 0, 'text': '.rst .pdf Welcome to Lan..."
1.0,110f550d-110b-4378-b95e-141397fa21bc,"[0.009401749819517136, 0.02443608082830906, 0....",None,"{'chunk': 1, 'text': 'Use Cases# Best practice..."
2.0,d5f00f02-3295-4567-b297-5e3262dc2728,"[-0.005517194513231516, 0.0208403542637825, 0....",None,"{'chunk': 2, 'text': 'Gallery: A collection of..."
3.0,0b6fe3c6-1f0e-4608-a950-43231e46b08a,"[-0.006499645300209522, 0.0011573900701478124,...",None,"{'chunk': 0, 'text': 'Search Error Please acti..."
4.0,39d5f15f-b973-42c0-8c9b-a2df49b627dc,"[-0.005658374633640051, 0.00817849114537239, 0...",None,"{'chunk': 0, 'text': '.md .pdf Dependents Depe..."


In [3]:
import os

use_serverless = os.environ.get("USE_SERVERLESS", "False").lower() == "true"

In [4]:
index_name = 'gpt-4-langchain-docs-fast'

In [9]:
!pip show pinecone-client

Name: pinecone-client
Version: 3.0.0
Summary: Pinecone client and SDK
Home-page: https://www.pinecone.io
Author: Pinecone Systems, Inc.
Author-email: support@pinecone.io
License: Apache-2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: certifi, tqdm, typing-extensions, urllib3
Required-by: pinecone-datasets


In [5]:
from pinecone import Pinecone

pc = Pinecone(api_key='YOUR_API')

In [6]:
from pinecone import Pinecone

# initialize connection to pinecone (get API key at app.pc.io)
api_key = os.environ.get('YOUR_API')

In [12]:
from pinecone import Pinecone

# initialize connection to pinecone (get API key at app.pc.io)
api_key = os.environ.get('YOUR_API') or 'YOUR_API'
environment = os.environ.get('gcp-starter')or 'gcp-starter'

# configure client
pc = Pinecone(api_key='YOUR_API')

In [13]:
from pinecone import ServerlessSpec, PodSpec

if use_serverless:
    cloud = os.environ.get('gcp-starter') or 'gcp-starter'
    spec = ServerlessSpec(cloud='aws', region='us-central1')
else:
    spec = PodSpec(environment='gcp-starter')

In [14]:
import time
from pinecone import Pinecone  # Ensure Pinecone is imported

# Initialize Pinecone client here if not already done

# Define your index name using lower-case characters
index_name = "pinecone-rag"

# Check if the index already exists
if index_name not in pc.list_indexes().names():
    # Create the index if it does not exist
    pc.create_index(
        index_name,
        dimension=1536,  # Dimensionality of the embeddings
        metric='cosine',  # Using cosine similarity
        spec=spec         # Your previously defined spec
    )
    print(f"Index '{index_name}' created.")
    # Wait for the index to be initialized
    time.sleep(1)
else:
    print(f"Index '{index_name}' already exists.")

# Connect to the index
index = pc.Index(index_name)
print(f"Connected to index '{index_name}'.")

# View index stats
index_stats = index.describe_index_stats()
print("Index Stats:", index_stats)


Index 'pinecone-rag' already exists.
Connected to index 'pinecone-rag'.
Index Stats: {'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}


In [16]:
for batch in dataset.iter_documents(batch_size=100):
    index.upsert(batch)

Retrievel

In [35]:
import openai

# get api key from platform.openai.com
openai.api_key = os.getenv('YOUR_API') or 'YOUR_API'

embed_model = "text-embedding-ada-002"

In [60]:
query = "What is Langchain?"

res = openai.Embedding.create(
    input=[query],
    engine=embed_model
)

# retrieve from Pinecone
xq = res['data'][0]['embedding']

# get relevant contexts (including the questions)
res = index.query(vector=xq, top_k=5, include_metadata=True)

In [55]:
res

{'matches': [{'id': 'ef0c5fb3-961a-4ede-88d1-d522fb5723ee',
              'metadata': {'chunk': 10.0,
                           'text': 'LangChain is an intuitive framework '
                                   'created to assist in developing '
                                   'applications driven by a language model, '
                                   'such as OpenAI or Hugging Face. Missing: '
                                   'decentralized | Must '
                                   'include:decentralized. LangChain, created '
                                   'by Harrison Chase, is a Python library '
                                   'that provides out-of-the-box support to '
                                   'build NLP applications using LLMs. '
                                   'Missing: decentralized | Must '
                                   'include:decentralized. LangChain provides '
                                   'a standard interface for chains, enabling '
 

Retrieval Augmented Generation

In [56]:
# get list of retrieved text
contexts = [item['metadata']['text'] for item in res['matches']]

augmented_query = "\n\n---\n\n".join(contexts)+"\n\n-----\n\n"+query

In [57]:
print(augmented_query)

LangChain is an intuitive framework created to assist in developing applications driven by a language model, such as OpenAI or Hugging Face. Missing: decentralized | Must include:decentralized. LangChain, created by Harrison Chase, is a Python library that provides out-of-the-box support to build NLP applications using LLMs. Missing: decentralized | Must include:decentralized. LangChain provides a standard interface for chains, enabling developers to create sequences of calls that go beyond a single LLM call. Chains ... Missing: decentralized platform natural. LangChain is a powerful framework that simplifies the process of building advanced language model applications. Missing: platform | Must include:platform. Are your language models ignoring previous instructions ... Duration: 32:23. Posted: Feb 21, 2023. LangChain is a framework that enables quick and easy development of applications ... Prompting is the new way of programming NLP models. Missing: decentralized platform. It then u

In [58]:
# system message to 'prime' the model
primer = f"""You are Q&A bot. A highly intelligent system that answers
user questions based on the information provided by the user above
each question. If the information can not be found in the information
provided by the user you truthfully say "I don't know".
"""

res = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": primer},
        {"role": "user", "content": augmented_query}
    ]
)

In [59]:
from IPython.display import Markdown

display(Markdown(res['choices'][0]['message']['content']))


LangChain is a software development framework designed to simplify the creation of applications using large language models (LLMs). It is written in Python and JavaScript and was initially released in October 2022. Developed by Harrison Chase, LangChain provides out-of-the-box support for developing applications powered by language models such as OpenAI or Hugging Face. This powerful tool can be used for a variety of applications including chatbots, generative question-answering, summarization, and much more. It connects to the AI models you want to use, abstracting the core building blocks of LLM applications and making it easier for developers to build LLM-powered applications. It is licensed under the MIT License.